In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import itertools
import time
from collections import Counter
import hdbscan
## get that here: https://github.com/scikit-learn-contrib/hdbscan

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn import metrics
from sklearn.metrics import classification_report

from sqlalchemy import create_engine
import datetime 
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib
pd.options.display.max_columns=300
pd.options.display.max_rows=100
# from TurbineTimeSeries.storage import MachineDataStore
plt.rcParams["figure.figsize"] = (14,8)
#from TurbineTimeSeries.transformations import PCA, StandardScaler, DropCols, DropSparseCols, LeftJoin
# %matplotlib inline

In [3]:
# from sklearn.cluster import AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering, DBSCAN
# from sklearn.mixture import GaussianMixture
# ## for agglomerative clustering, give linkage : {“ward”, “complete”, “average”}

# n_clusts = 3
# kmeans = KMeans(init='k-means++', n_clusters=n_clusts, n_init=10)
# AffinityProp = AffinityPropagation()
# Meanshift = MeanShift(n_jobs=7)
# Spectral = SpectralClustering(n_clusters=n_clusts, affinity='nearest_neighbors',n_jobs=7)
# Agglom_ward = AgglomerativeClustering(n_clusters=n_clusts, linkage='ward')
# Agglom_complete = AgglomerativeClustering(n_clusters=n_clusts, linkage='complete')
# Agglom_avg = AgglomerativeClustering(n_clusters=n_clusts, linkage='average')
# Dbscan = DBSCAN(eps=5, min_samples=n_clusts)
# GMM_spherical = GaussianMixture(n_components=n_clusts, covariance_type='spherical' )
# GMM_diag = GaussianMixture(n_components=n_clusts, covariance_type='diag' )
# GMM_tied = GaussianMixture(n_components=n_clusts, covariance_type='tied' )
# GMM_full = GaussianMixture(n_components=n_clusts, covariance_type='full' )



# clustering_algo_dict = {
#                         'kmeans':kmeans, 
# #                         'AffinityProp': AffinityProp, #mem errors out. affinity propogation calculates full distance matrix so it's quadratic memory required. 10k samples would use ~80gb ram.
#                         'Meanshift':Meanshift,  #runs forever
#                         'Spectral':Spectral, ## runs forever
# #                         'Agglom_ward':Agglom_ward, ## uses too much ram
# #                         'Agglom_complete':Agglom_complete,
# #                         'Agglom_avg':Agglom_avg,
#                        'Dbscan':Dbscan,
#                         'GMM_spherical':GMM_spherical,
#                         'GMM_diag':GMM_diag, 
#                         'GMM_tied':GMM_tied,
#                         'GMM_full':GMM_full
#                        }

In [2]:
# model_number = 2

# store = MachineDataStore('.config')

# # model_data_hr = (store.query(model_number,'1hr')
# #                  .not_null(['timestamp','psn'])
# # #                  .exclude_psn([44,52,54,70])
# #                  .execute())

# model_data_min = (store.query(model_number,'10min')
#                   .not_null(['timestamp','psn'])
#                   .exclude_psn([44,52,54,70])
#                   .execute())

# model_data_min.head()
# model1_1hr = pd.read_csv('../../../data/raw_data_model1.csv',index_col=0)
# model2_1hr = pd.read_csv('../../../data/raw_data_model2.csv',index_col=0)

# model1_10min = pd.read_csv('../../../data/raw_data_model1_10min.csv',index_col=0)
model2_10min = pd.read_csv('../../../data/raw_data_model2_10min.csv')#,index_col=0)

## convert all to timestamps
# model1_1hr['timestamp'] = model1_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model2_1hr['timestamp'] = model2_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model1_10min['timestamp'] = model1_10min['timestamp'].apply(lambda x: pd.Timestamp(x))
model2_10min['timestamp'] = model2_10min['timestamp'].apply(lambda x: pd.Timestamp(x))


# print('Shape of model1 1hr data: ', model1_1hr.shape)
# print('Shape of model2 1hr data: ', model2_1hr.shape)
# print('Shape of model1 10min data: ', model1_10min.shape)
print('Shape of model2 10min data: ', model2_10min.shape)

Shape of model2 10min data:  (1602326, 76)


In [3]:
model2_10min=model2_10min.sort_values(by=['psn','timestamp'],ascending=(True,True))
model2_10min=model2_10min.dropna()

In [4]:
data_dictionary = pd.read_csv('data_dictionary_model2.csv')
# data_dictionary

In [5]:
# [type(model2_10min.head()['timestamp'][0]) ## should be timestamp]
[i for i in data_dictionary['COLUMN_NAME'].str.lower().values if i not in model2_10min.columns.unique()]

['lo_c_brg1']

In [6]:
subsystem_dict = {str(i).lower(): list(data_dictionary[data_dictionary['SUBSYSTEM']==i]['COLUMN_NAME'].str.lower().values) for i in data_dictionary['SUBSYSTEM'].unique()}
del subsystem_dict['summary'] ## this group pretty useless

In [7]:
print(list(subsystem_dict.keys()))

['gas path', 'fuel', 'generator', 'vibration', 'lube oil system', 'enclosure', 'package equipment']


In [8]:
for i in subsystem_dict.keys():
    print(i,len(subsystem_dict[i]))

gas path 29
fuel 12
generator 12
vibration 6
lube oil system 8
enclosure 2
package equipment 1


In [9]:
# model2_10min.isnull().sum().sort_values()

In [10]:
def subsystem_pca(df,subsystem_dict,subsystems = [], psns = 'all'):
    
    subset = []
    if isinstance(subsystems,list)==True:
        if len(subsystems) > 0:
            for i in subsystems:
                subset = subset + subsystem_dict[i] ## populate subset with list of columns
        else:
            print('Empty list of subsystems detected. Using all subsystems for PCA')
            subset = [s for L in subsystem_dict.keys() for s in subsystem_dict[L]]
    else: 
        raise Exception('Please ensure subsystems parameter is a list')
        
    if psns == 'all' or isinstance(psns, (list, int))==True:
        pass
    else:
        raise Exception('Please provide a list of psns, single psn, or "all"')
    


    available_subset = [i for i in subset if i in df.columns.values]
    if len(available_subset)<len(subset):
        print('excluding columns: ', list(set(subset)-set(available_subset)))
    available_subset =  available_subset + ['id','timestamp','psn']
    model_data = df[available_subset]
    
    
    
    skipped_cols = ['sum_esn','sum_eng_st', 'sum_eng_h']
    index_cols = ['id','timestamp','psn']
    data_cols = [c for c in model_data.columns if (c not in index_cols) and (c not in skipped_cols)]
    
    missing_values = model_data.isnull().sum().sort_values()
    sparse_cols = [x for x in missing_values.index if missing_values[x] > 30000]
    clean_data_cols = [x for x in data_cols if x not in sparse_cols]
    data = model_data[index_cols + clean_data_cols].dropna()#.reset_index()
    print(data.shape)
    clean_data = StandardScaler().fit_transform(data[clean_data_cols])

    pca =  PCA().fit(clean_data)
    reduced = pca.transform(clean_data)
    reduced_df = pd.DataFrame(reduced)
    reduced_df['psn'] = data.psn.values
    reduced_df['timestamp'] = data.timestamp.values
    return(reduced_df)

In [11]:
def plot_eigs_subplots(reduced_df, n_eigs_x, n_eigs_y, psns, savefig = False,path=None,figname = None):
    if isinstance(n_eigs_x,int) == True:
        pass
    else:
        raise Exception('n_eigs_x must be an integer')
        
    if isinstance(n_eigs_y,int) == True:
        pass
    else:
        raise Exception('n_eigs_y must be an integer')
    

    
    if psns == 'all':
#         fig_base_title = 'All psns'
        pass
    elif isinstance(psns, list)==True:
        model2_10min[model2_10min['psn'].isin(psns)]
#         fig_base_title = 'PSN ' + str(psns)
        
    elif isinstance(psns,int) == True:
        reduced_df = reduced_df[reduced_df['psn']==psns]
#         fig_base_title = 'PSN ' + str(psns)
    else:
        raise Exception('Please provide a list of psns, single psn, or "all"')
        
    if path == None:
        path = ''
    else:
        pass

        
    f, axarr = plt.subplots(n_eigs_x, n_eigs_y)
    f.set_figheight(40)
    f.set_figwidth(40)
    f.suptitle(path + fname,fontsize=16)
    for i in range(n_eigs_x):
        for j in range(n_eigs_y):

            if i==j:
                continue
            axarr[i, j].scatter(reduced_df[i].values,reduced_df[j].values,3,alpha=0.5)
            axarr[i, j].set_title('Eig '+str(i) + ' vs Eig '+ str(j))
    if savefig == False:
        plt.show()
    else:
        
        f.savefig(path+str('\\') + fname +'.png')      
#         f.suptitle(path + fig_base_title + ' ' + str(n_eigs_x) + 'x' + str(n_eigs_y) + ' subplots',fontsize=16)
#         f.savefig(path + fig_base_title + ' ' + str(n_eigs_x) + 'x' + str(n_eigs_y) + ' subplots.png')
#         plt.show()
        plt.close()

## looping through potential subsystems/tags to look for clusters in eigs

In [12]:
color_pal = ['#33a02c', '#1f78b4', '#ff7f00', '#a6cee3','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#cab2d6','#6a3d9a']

In [13]:
model2_10min.dropna().shape

(1602281, 76)

In [14]:
rd_df = subsystem_pca(model2_10min,subsystem_dict, [], psns = 'all')
print(rd_df.shape)
rd_df['id'] = model2_10min['id']

Empty list of subsystems detected. Using all subsystems for PCA
excluding columns:  ['lo_c_brg1']
(1602281, 72)
(1602281, 71)


In [15]:
for i in sorted(rd_df['psn'].unique()):
    print(i, rd_df[rd_df['psn']==i].shape )

34 (103217, 72)
35 (71208, 72)
36 (70287, 72)
37 (65650, 72)
38 (30025, 72)
39 (22333, 72)
40 (14024, 72)
41 (15484, 72)
42 (95461, 72)
45 (48741, 72)
46 (7592, 72)
47 (10941, 72)
48 (92989, 72)
49 (89536, 72)
50 (2516, 72)
51 (5462, 72)
53 (36714, 72)
55 (77340, 72)
56 (80596, 72)
57 (65672, 72)
58 (58907, 72)
59 (66452, 72)
60 (27075, 72)
61 (17242, 72)
62 (27185, 72)
63 (1331, 72)
64 (56204, 72)
65 (51089, 72)
66 (59654, 72)
67 (57318, 72)
68 (82581, 72)
69 (12866, 72)
71 (22779, 72)
72 (55810, 72)


In [16]:
def hdbscan_eig_clusterer(reduced_df, cols, cluster_algo, cluster_params, psn='all'):
    """
    reduced_df: dataframe output from a PCA
    doesn't matter what index comes in, that's what will go out anywaysaaaaaa
    
    """
    ## todo: index by 
    returndf = pd.DataFrame()
    psns = []
    if psn == 'all':
        psns = sorted(reduced_df['psn'].unique())
    else:
        psns = psn
    
    for pkg in psns:
        tempdf = reduced_df[reduced['psn']==pkg]
        min_clust_size = int(len(newdf)/70.3)+1
        clusterer = cluster_algo(min_cluster_size=min_clust_size)
        clusterer = clusterer.fit(tempdf[cols])
        results = clusterer.predict(tempdf[cols])
        tempdf['cluster'] = results
        returndf = returndf.append(tempdf)
    return(reduced_df)

def plot_clusters(cluster_df, x, y, colorpal, save_fig = False, directory=None, title=''):
    plt.figure(figsize=(28,16))
    for i in sorted(cluster_df['cluster'].unique()):
        tempdf = cluster_df[cluster_df['cluster']==i]
        plt.scatter(tempdf[x].values,tempdf[y].values,s=3,alpha=0.5,c=color_pal[i],label = 'cluster '+str(i))
    plt.xlabel('Eigenvector '+str(x))
    plt.ylabel('Eigenvector '+str(y))
    fulltitle = 'Eig '+str(x) + ' vs Eig ' + str(y)+' '+ str(title)#' Clustering using HDBSCAN 20eigs')
    plt.title(title)
    plt.legend()
    if save_fig == True:
        if directory != None:
            plt.savefig(directory + title + 'png')
        else:
            plt.savefig(title+'.png')
        plt.show()
        plt.close()
    else:
        plt.show()


### make sure final code returns df with psn,timestamp index and cluster num col

In [17]:
# num_eigs_toclust = 20 ## 20 eigenvectors cover 90% variance

# current_directory = os.getcwd()
# final_directory = os.path.join(current_directory,r'Hdbscan Clustering mc lendiv70')
# if not os.path.isdir(final_directory):
#     os.mkdir(final_directory)

# # for psn in sorted(rd_df['psn'].unique()):
# # def clusterandplot(rd_df, psn, num_eigs_toclust):
# # for psn in sorted(rd_df['psn'].unique()):
# for psn in [35,37,40,45,46,47,49,55,57,58,62]:
#     print('started: ',psn)
#     nao = time.time()
#     newdf = rd_df[rd_df['psn']==psn]
#     min_clust_size = int(len(newdf)/70.3)+1
# #     hdbscan = hdbscan.HDBSCAN(min_cluster_size=min_clust_size) ## 70.3 chosen arbitrarily
#     print('    clustering with min_cluster_size={} min_samples={}.....'.format(min_clust_size,int(min_clust_size)),end="",flush=True)
# #     cluster_results = cluster_eigs(reduced_df=rd_df, psn=psn, cols=list(range(num_eigs_toclust)), cluster_algo=hdbscan)
#     clusterer = hdbscan.HDBSCAN(min_cluster_size=min_clust_size, min_samples=int(min_clust_size))
#     clusterer_results = clusterer.fit_predict(newdf[list(range(num_eigs_toclust))])
#     print(newdf.shape, len(clusterer_results))
#     print('finished in {} seconds'.format(time.time()-nao))
#     newdf['cluster'] = clusterer_results
#     figname = 'PSN '+str(psn)+'HDBSCAN Clustering 20eigs minsize='+str(min_clust_size)
#     print('    saving figure.....',end="",flush=True)
#     plot_clusters(newdf, x=0, y=1, colorpal = color_pal, save_fig=True, directory=final_directory+'\\', title = figname)
#     print('finished in total {} seconds'.format(time.time()-nao))


In [18]:
def find_power_step (df,powercol,jump = 0.2):
    ## function will return another dataframe where the power columns is replace with 1s and 0s. 1 represents an outlier.
    ## jumps is a percentage
    ## assumes data coming in is sorted by psn by timestamp.

    
    df = df[[powercol,'timestamp','psn']]

    df = df.where((pd.notnull(df)),np.nan) ## replaces nulls with nans for math stuffs.
    df2 = pd.DataFrame(index = df.index.values[1:],columns = df.columns.values)
    df2['timestamp'] = df['timestamp'].values[1:]
    ## np.divide will divide the first parameter by the second parameter so the resulting series starts
    ## from the original dataframe's 1st item, not the 0th item.
#     shifted = pd.Series(np.subtract(df[powercol].values[1:],df[powercol].values[:-1]),index=df.index.values[:-1])

    shifted = pd.Series(np.divide(df[powercol].values[:-1],df[powercol].values[1:]),index=df.index.values[1:])-1

    ## take all values and subtract from previous values. if unchanging, then result will be 0
    ## create numpy array of all False

    tomap = np.zeros(len(df2),dtype=int)  
    
    
    ## find where jumps in data are greater than given jump parameter, and set numpy array equal to True in those positions
    for j in shifted[abs(shifted)>=jump].index.values:
        ## for now just doing where power jumps.
        ## does not catch all in betweens since transients are only 10-20 mins long.
        if (df.loc[j]['timestamp'] - df.loc[j-1]['timestamp']) > pd.Timedelta('12 minutes'): ## if data is not continuous
            continue
                        
        else:
            tomap[j] = True


    
    ## map back numpy array to df2
    df2[powercol] = pd.Series(tomap).loc[1:]

    df2.loc[0] = [0,df.loc[0]['timestamp'],0] ## assume not outlier for row 0. this also 0-indexes the dataframe again
#     df2 = df2.sort_index()
    df2['psn'] = df['psn']
    return(df2)


### validate different cluster parameters using powerstep function 

In [19]:
model2_10min_powerstep = find_power_step(model2_10min,'perf_pow',jump=0.3)
model2_10min['powerjump'] = model2_10min_powerstep['perf_pow']
model2_10min['powerjump'] = model2_10min['powerjump'].fillna(0).astype(int)
model2_10min_powerstep[model2_10min_powerstep['perf_pow']==1].shape

(5376, 3)

In [20]:
len(model2_10min_powerstep)

1602280

In [21]:
Counter(model2_10min['powerjump'])

Counter({0: 1596905, 1: 5376})

In [22]:
cluster_sizes = [110,120]#[40,50,60,70,80,90,100,110,120]#[10,20,30,40,50,60,70,80,90,100,110]
min_sample_sizes = [0.2,0.4,0.6,0.8,1,1.2,1.4]
psns = sorted(list(set(rd_df['psn'].values)))#[35,37,40,45,46,47,49,55,57,58,62]

mindex0 = list(sorted(itertools.chain(*[range(len(cluster_sizes))for i in min_sample_sizes])))
mindex1 = list(itertools.chain(*[range(len(min_sample_sizes))for i in cluster_sizes]))

my_index = pd.MultiIndex(levels=[cluster_sizes,min_sample_sizes],
                             labels=[mindex0,mindex1],
                             names=[u'min_cluster_size', u'min_samples'])


# transients_scores_df = pd.DataFrame(index=my_index,columns=psns)
# normals_scores_df = pd.DataFrame(index=my_index,columns=psns)
## slice df like this:  testdf.loc[10].loc[0.1][35]




num_eigs_toclust = 20

for clust_sizer in cluster_sizes:
    transients_scores_df = pd.DataFrame(index=my_index,columns=psns)
    normals_scores_df = pd.DataFrame(index=my_index,columns=psns)
    for sample_sizer in min_sample_sizes:
        for psn in psns:
            print('clust_sizer: {}, sample_sizer: {}, psn: {}'.format(clust_sizer,sample_sizer,psn))

            powerjump_validation = model2_10min[model2_10min['psn']==psn]['powerjump'].values
            newdf = rd_df[rd_df['psn']==psn]
            min_clust_size = int(len(newdf)/clust_sizer)+1
            clusterer = hdbscan.HDBSCAN(min_cluster_size=min_clust_size, min_samples=int(min_clust_size*sample_sizer),core_dist_n_jobs=-1)
            clusterer_results = clusterer.fit_predict(newdf[list(range(num_eigs_toclust))])
            preds = [1 if i==-1 else 0 for i in clusterer_results]
            
            idxs_of_ones = []
            for i,j in enumerate(powerjump_validation):
                if j == 1:
                    idxs_of_ones.append(i)
            transient_validation = [powerjump_validation[i] for i in idxs_of_ones]
            transient_predictions = [preds[i] for i in idxs_of_ones]
            print('number of powerjumps: ', len(transient_validation))
            print('number of noise by cluster: ', Counter(transient_predictions)[1])
            f1score = metrics.f1_score(transient_validation, transient_predictions,average='micro')
            ## use micro to take into account false pos/neg
            transients_scores_df.loc[clust_sizer].loc[sample_sizer][psn]= f1score
            
            idxs_of_zeroes = []
            for i,j in enumerate(powerjump_validation):
                if j == 0:
                    idxs_of_zeroes.append(i)
            normals_validation = [powerjump_validation[i] for i in idxs_of_zeroes]
            normals_predictions = [preds[i] for i in idxs_of_zeroes]
            f1score_norm = metrics.f1_score(normals_validation, normals_predictions,average='micro')
            normals_scores_df.loc[clust_sizer].loc[sample_sizer][psn]= f1score_norm
    print('saving files')
    transients_scores_df.to_csv('transients_scores_clust_'+str(clust_sizer)+'.csv')
    normals_scores_df.to_csv('normals_scores_clust_'+str(clust_sizer)+'.csv')        
    print('done')        
            
            
# newdf['cluster'] = clusterer_results
## min_clust size/70 gives 256 "noise" datapoints
## min clust size/40 gives  "noise" datapoints

clust_sizer: 110, sample_sizer: 0.2, psn: 34
number of powerjumps:  52
number of noise by cluster:  44
clust_sizer: 110, sample_sizer: 0.2, psn: 35
number of powerjumps:  635
number of noise by cluster:  606
clust_sizer: 110, sample_sizer: 0.2, psn: 36
number of powerjumps:  546
number of noise by cluster:  512
clust_sizer: 110, sample_sizer: 0.2, psn: 37
number of powerjumps:  94
number of noise by cluster:  9
clust_sizer: 110, sample_sizer: 0.2, psn: 38
number of powerjumps:  10
number of noise by cluster:  10
clust_sizer: 110, sample_sizer: 0.2, psn: 39
number of powerjumps:  15
number of noise by cluster:  12
clust_sizer: 110, sample_sizer: 0.2, psn: 40
number of powerjumps:  52
number of noise by cluster:  2
clust_sizer: 110, sample_sizer: 0.2, psn: 41
number of powerjumps:  34
number of noise by cluster:  3
clust_sizer: 110, sample_sizer: 0.2, psn: 42
number of powerjumps:  725
number of noise by cluster:  5
clust_sizer: 110, sample_sizer: 0.2, psn: 45
number of powerjumps:  36
n

number of powerjumps:  178
number of noise by cluster:  64
clust_sizer: 110, sample_sizer: 0.6, psn: 49
number of powerjumps:  206
number of noise by cluster:  58
clust_sizer: 110, sample_sizer: 0.6, psn: 50
number of powerjumps:  16
number of noise by cluster:  0
clust_sizer: 110, sample_sizer: 0.6, psn: 51
number of powerjumps:  21
number of noise by cluster:  1
clust_sizer: 110, sample_sizer: 0.6, psn: 53
number of powerjumps:  47
number of noise by cluster:  44
clust_sizer: 110, sample_sizer: 0.6, psn: 55
number of powerjumps:  90
number of noise by cluster:  0
clust_sizer: 110, sample_sizer: 0.6, psn: 56
number of powerjumps:  100
number of noise by cluster:  18
clust_sizer: 110, sample_sizer: 0.6, psn: 57
number of powerjumps:  10
number of noise by cluster:  0
clust_sizer: 110, sample_sizer: 0.6, psn: 58
number of powerjumps:  5
number of noise by cluster:  4
clust_sizer: 110, sample_sizer: 0.6, psn: 59
number of powerjumps:  233
number of noise by cluster:  232
clust_sizer: 110

number of powerjumps:  440
number of noise by cluster:  64
clust_sizer: 110, sample_sizer: 1, psn: 63
number of powerjumps:  1
number of noise by cluster:  0
clust_sizer: 110, sample_sizer: 1, psn: 64
number of powerjumps:  200
number of noise by cluster:  60
clust_sizer: 110, sample_sizer: 1, psn: 65
number of powerjumps:  206
number of noise by cluster:  38
clust_sizer: 110, sample_sizer: 1, psn: 66
number of powerjumps:  272
number of noise by cluster:  119
clust_sizer: 110, sample_sizer: 1, psn: 67
number of powerjumps:  223
number of noise by cluster:  79
clust_sizer: 110, sample_sizer: 1, psn: 68
number of powerjumps:  48
number of noise by cluster:  48
clust_sizer: 110, sample_sizer: 1, psn: 69
number of powerjumps:  35
number of noise by cluster:  35
clust_sizer: 110, sample_sizer: 1, psn: 71
number of powerjumps:  62
number of noise by cluster:  13
clust_sizer: 110, sample_sizer: 1, psn: 72
number of powerjumps:  68
number of noise by cluster:  10
clust_sizer: 110, sample_size

number of powerjumps:  546
number of noise by cluster:  521
clust_sizer: 120, sample_sizer: 0.2, psn: 37
number of powerjumps:  94
number of noise by cluster:  9
clust_sizer: 120, sample_sizer: 0.2, psn: 38
number of powerjumps:  10
number of noise by cluster:  10
clust_sizer: 120, sample_sizer: 0.2, psn: 39
number of powerjumps:  15
number of noise by cluster:  12
clust_sizer: 120, sample_sizer: 0.2, psn: 40
number of powerjumps:  52
number of noise by cluster:  2
clust_sizer: 120, sample_sizer: 0.2, psn: 41
number of powerjumps:  34
number of noise by cluster:  3
clust_sizer: 120, sample_sizer: 0.2, psn: 42
number of powerjumps:  725
number of noise by cluster:  1
clust_sizer: 120, sample_sizer: 0.2, psn: 45
number of powerjumps:  36
number of noise by cluster:  4
clust_sizer: 120, sample_sizer: 0.2, psn: 46
number of powerjumps:  14
number of noise by cluster:  8
clust_sizer: 120, sample_sizer: 0.2, psn: 47
number of powerjumps:  27
number of noise by cluster:  9
clust_sizer: 120, s

number of powerjumps:  16
number of noise by cluster:  0
clust_sizer: 120, sample_sizer: 0.6, psn: 51
number of powerjumps:  21
number of noise by cluster:  1
clust_sizer: 120, sample_sizer: 0.6, psn: 53
number of powerjumps:  47
number of noise by cluster:  44
clust_sizer: 120, sample_sizer: 0.6, psn: 55
number of powerjumps:  90
number of noise by cluster:  0
clust_sizer: 120, sample_sizer: 0.6, psn: 56
number of powerjumps:  100
number of noise by cluster:  18
clust_sizer: 120, sample_sizer: 0.6, psn: 57
number of powerjumps:  10
number of noise by cluster:  0
clust_sizer: 120, sample_sizer: 0.6, psn: 58
number of powerjumps:  5
number of noise by cluster:  4
clust_sizer: 120, sample_sizer: 0.6, psn: 59
number of powerjumps:  233
number of noise by cluster:  232
clust_sizer: 120, sample_sizer: 0.6, psn: 60
number of powerjumps:  447
number of noise by cluster:  42
clust_sizer: 120, sample_sizer: 0.6, psn: 61
number of powerjumps:  228
number of noise by cluster:  54
clust_sizer: 120

number of powerjumps:  200
number of noise by cluster:  59
clust_sizer: 120, sample_sizer: 1, psn: 65
number of powerjumps:  206
number of noise by cluster:  35
clust_sizer: 120, sample_sizer: 1, psn: 66
number of powerjumps:  272
number of noise by cluster:  115
clust_sizer: 120, sample_sizer: 1, psn: 67
number of powerjumps:  223
number of noise by cluster:  78
clust_sizer: 120, sample_sizer: 1, psn: 68
number of powerjumps:  48
number of noise by cluster:  48
clust_sizer: 120, sample_sizer: 1, psn: 69
number of powerjumps:  35
number of noise by cluster:  35
clust_sizer: 120, sample_sizer: 1, psn: 71
number of powerjumps:  62
number of noise by cluster:  12
clust_sizer: 120, sample_sizer: 1, psn: 72
number of powerjumps:  68
number of noise by cluster:  7
clust_sizer: 120, sample_sizer: 1.2, psn: 34
number of powerjumps:  52
number of noise by cluster:  43
clust_sizer: 120, sample_sizer: 1.2, psn: 35
number of powerjumps:  635
number of noise by cluster:  635
clust_sizer: 120, sampl

In [23]:
# transients_scores_df.to_csv('transients_scores_full.csv')

In [ ]:
# normals_scores_df.to_csv('normals_scores_full.csv')

## todo: join dfs together

In [27]:
import glob

In [43]:
tran_tojoin = glob.glob('transients_scores_clust*')
transients_scores_df = pd.DataFrame()
for i in tran_tojoin:
    transients_scores_df = transients_scores_df.append(pd.read_csv(i))
transients_scores_df = transients_scores_df.dropna().set_index(['min_cluster_size','min_samples']).sort_index()

In [45]:
transients_scores_df.head(7)

34   35        36        37   38   39  \
min_cluster_size min_samples                                                
40               0.2          0.923077  1.0  0.967033  0.819149  1.0  0.8   
                 0.4          0.826923  1.0  1.000000  0.851064  1.0  0.8   
                 0.6          0.846154  1.0  1.000000  0.882979  1.0  0.8   
                 0.8          0.846154  1.0  1.000000  0.893617  1.0  0.8   
                 1.0          0.826923  1.0  1.000000  0.851064  1.0  0.8   
                 1.2          0.807692  1.0  1.000000  0.872340  1.0  0.8   
                 1.4          0.769231  1.0  1.000000  0.925532  1.0  0.8   

                                    40        41   42        45        46  \
min_cluster_size min_samples                                                
40               0.2          0.096154  0.176471  1.0  0.972222  0.142857   
                 0.4          0.115385  0.205882  1.0  0.972222  0.214286   
                 0.6          0.134615  0.205882  1.0  0.972222  0.214286   
                 0.8          0.153846  0.205882  1.0  0.972222  0.214286   
                 1.0          0.173077  0.205882  1.0  0.972222  0.214286   
                 1.2          0.173077  0.205882  1.0  0.972222  0.000000   
                 1.4          0.173077  0.205882  1.0  0.972222  0.071429   

                                    47        48        49      50        51  \
min_cluster_size min_samples                                                   
40               0.2          0.111111  0.556180  0.233010  0.0000  0.047619   
                 0.4          0.148148  0.410112  0.174757  0.0000  0.047619   
                 0.6          0.000000  0.398876  0.237864  0.0000  0.095238   
                 0.8          0.000000  0.286517  0.237864  0.0000  0.095238   
                 1.0          0.185185  0.325843  0.257282  0.0625  0.095238   
                 1.2          0.185185  0.359551  0.378641  0.0625  0.095238   
                 1.4          0.222222  0.365169  0.388350  0.0625  0.095238   

                                    53        55    56   57   58        59  \
min_cluster_size min_samples                                                 
40               0.2          0.936170  0.966667  0.97  1.0  0.8  0.995708   
                 0.4          0.936170  0.988889  0.96  1.0  0.8  0.995708   
                 0.6          0.936170  0.977778  0.97  1.0  0.8  1.000000   
                 0.8          0.957447  0.977778  0.98  1.0  0.8  1.000000   
                 1.0          0.957447  0.977778  0.97  1.0  0.8  1.000000   
                 1.2          0.957447  0.977778  0.98  1.0  0.8  1.000000   
                 1.4          0.978723  0.977778  0.98  1.0  0.8  1.000000   

                                    60        61   62   63     64        65  \
min_cluster_size min_samples                                                  
40               0.2          0.093960  0.236842  1.0  1.0  0.245  0.155340   
                 0.4          0.219239  0.157895  1.0  1.0  0.310  0.184466   
                 0.6          0.228188  0.333333  1.0  1.0  0.365  0.194175   
                 0.8          0.230425  0.179825  1.0  1.0  0.400  0.247573   
                 1.0          0.237136  0.201754  1.0  1.0  0.430  0.276699   
                 1.2          0.241611  0.407895  1.0  1.0  0.470  0.344660   
                 1.4          0.239374  0.214912  1.0  1.0  0.490  0.286408   

                                    66        67        68   69        71   72  
min_cluster_size min_samples                                                    
40               0.2          0.360294  0.291480  1.000000  1.0  0.161290  1.0  
                 0.4          0.448529  0.363229  1.000000  1.0  0.209677  1.0  
                 0.6          0.433824  0.385650  1.000000  1.0  0.274194  1.0  
                 0.8          0.488971  0.434978  0.979167  1.0  0.290323  1.0  
                 1.0      

In [46]:
mylist = []
for col in transients_scores_df.columns:
    print(col,transients_scores_df[col].max())
    ## grabs the rows where f1 score is max
    mylist.append(transients_scores_df[transients_scores_df[col]==transients_scores_df[col].max()][col])
print(Counter([x[1] for ii in mylist for x in ii.index.values ])) ## counts number of times that the min_samples multipler appears
print(Counter([x[0] for ii in mylist for x in ii.index.values ]))

34 0.9230769230769232
35 1.0
36 1.0
37 0.9255319148936171
38 1.0
39 0.8000000000000002
40 0.17307692307692307
41 0.2058823529411765
42 1.0
45 0.9722222222222222
46 0.5714285714285714
47 0.4814814814814815
48 0.702247191011236
49 0.3883495145631068
50 0.0625
51 0.09523809523809523
53 0.9787234042553192
55 1.0
56 0.98
57 1.0
58 0.8000000000000002
59 1.0
60 0.2460850111856823
61 0.8026315789473685
62 1.0
63 1.0
64 0.49
65 0.3446601941747573
66 0.5294117647058824
67 0.4887892376681615
68 1.0
69 1.0
71 0.3709677419354839
72 1.0
Counter({1.4: 140, 1.2: 124, 1.0: 108, 0.8: 97, 0.4: 90, 0.6: 89, 0.2: 80})
Counter({40: 119, 50: 109, 60: 93, 70: 82, 80: 76, 90: 70, 100: 66, 110: 60, 120: 53})


In [50]:
norm_tojoin = glob.glob('normals_scores_clust*')
normals_scores_df = pd.DataFrame()
for i in norm_tojoin:
    normals_scores_df = normals_scores_df.append(pd.read_csv(i))
normals_scores_df = normals_scores_df.dropna().set_index(['min_cluster_size','min_samples']).sort_index()

In [51]:
normals_scores_df

34        35        36        37  \
min_cluster_size min_samples                                           
40               0.2          0.763331  0.000000  0.287378  0.711087   
                 0.4          0.825435  0.000000  0.000000  0.620081   
                 0.6          0.770077  0.000000  0.000000  0.501388   
                 0.8          0.738157  0.000000  0.000000  0.460965   
                 1.0          0.713992  0.000000  0.000000  0.532934   
                 1.2          0.732458  0.000000  0.000000  0.474632   
                 1.4          0.747356  0.000000  0.000000  0.376289   
50               0.2          0.774100  0.074079  0.329964  0.993380   
                 0.4          0.839984  0.000000  0.152235  0.641147   
                 0.6          0.810158  0.000000  0.220631  0.600494   
                 0.8          0.764523  0.000000  0.000000  0.501922   
                 1.0          0.737993  0.000000  0.000000  0.460949   
                 1.2          0.726070  0.000000  0.000000  0.499008   
                 1.4          0.733223  0.000000  0.000000  0.487659   
60               0.2          0.808559  0.085571  0.393771  0.993563   
                 0.4          0.852295  0.000000  0.237622  0.993212   
                 0.6          0.823477  0.000000  0.129465  0.992876   
                 0.8          0.786720  0.000000  0.121378  0.991763   
                 1.0          0.761751  0.000000  0.000000  0.490100   
                 1.2          0.737993  0.000000  0.000000  0.460949   
                 1.4          0.726807  0.000000  0.000000  0.456373   
70               0.2          0.831464  0.078968  0.439842  0.994524   
                 0.4          0.864014  0.000000  0.261525  0.993303   
                 0.6          0.833946  0.000000  0.270415  0.993090   
                 0.8          0.815616  0.000000  0.222538  0.992434   
                 1.0          0.771279  0.000000  0.127185  0.991320   
                 1.2          0.760423  0.000000  0.118696  0.989703   
                 1.4          0.737993  0.000000  0.000000  0.460965   
80               0.2          0.847836  0.084026  0.458009  0.996964   
                 0.4          0.762701  0.061270  0.287378  0.993364   
                 0.6          0.843862  0.055021  0.200298  0.993151   
                 0.8          0.823477  0.000000  0.129465  0.992892   
                 1.0          0.799438  0.000000  0.117191  0.992159   
                 1.2          0.765851  0.000000  0.131300  0.990893   
                 1.4          0.758329  0.000000  0.118424  0.989292   
90               0.2          0.856967  0.473524  0.478886  0.997132   
                 0.4          0.787001  0.069120  0.299953  0.993303   
                 0.6          0.852295  0.054511  0.237622  0.993212   
                 0.8          0.831745  0.058478  0.278846  0.993029   
                 1.0          0.816856  0.000000  0.225205  0.992571   
                 1.2          0.786720  0.000000  0.121378  0.991763   
                 1.4          0.761799  0.000000  0.131429  0.990573   
100              0.2          0.870169  0.500829  0.531839  0.997270   
                 0.4          0.774100  0.074079  0.329964  0.993380   
                 0.6          0.859730  0.045839  0.246326  0.993303   
                 0.8          0.838666  0.052428  0.152235  0.993136   
                 1.0          0.823399  0.061355  0.129465  0.992876   
                 1.2          0.807183  0.000000  0.220631  0.992297   
                 1.4          0.776184  0.000000  0.127558  0.991564   
110              0.2          0.881278  0.483159  0.514016  0.997331   
                 0.4          0.793670  0.082369  0.354526  0.993441   
                 0.6          0.867659  0.052584  0.277082  0.993303   
                 0.8          0.845946  0.060590  0.209819  0.993181   
                 1.0          0.830253  0.061624  0.281785  0.993029   
    

In [52]:
mylist = []
for col in normals_scores_df.columns:
    print(col,normals_scores_df[col].max())
    ## grabs the rows where f1 score is max
    mylist.append(normals_scores_df[normals_scores_df[col]==normals_scores_df[col].max()][col])
print(Counter([x[1] for ii in mylist for x in ii.index.values ])) ## counts number of times that the min_samples multipler appears
print(Counter([x[0] for ii in mylist for x in ii.index.values ]))

34 0.8825667619832307
35 0.5008289289104898
36 0.5318392337362527
37 0.9973457807065714
38 0.9800099950024987
39 0.998252531588852
40 0.998997995991984
41 0.9988996763754046
42 0.9999894443506164
45 0.9984806488040242
46 0.997492742148324
47 0.9997251236943376
48 0.9893008371852474
49 0.9945818873838576
50 0.9976
51 0.9987134717882742
53 0.7740475086590121
55 0.999663430420712
56 0.9995527728085868
57 0.9999238524565196
58 0.9998302264778786
59 0.9928872378018394
60 0.9977842872164638
61 0.991653932055954
62 0.9933819405496356
63 1.0
64 0.9965716734518963
65 0.9964428198022915
66 0.9926913879626824
67 0.9972852263770908
68 0.8252456593120328
69 0.8639233107318214
71 0.9987674428841836
72 0.9991927092676977
Counter({0.2: 57, 0.4: 22, 0.6: 18, 0.8: 18, 1.0: 16, 1.2: 15, 1.4: 12})
Counter({120: 34, 110: 22, 100: 21, 90: 19, 70: 16, 80: 15, 60: 13, 40: 10, 50: 8})
